## Tabular training

In [55]:
# Install libraries on first run
#! pip install -q ipynb fastai pathlib pandas

To illustrate the tabular application, we will use the example of the Adult dataset where we have to predict if a person is earning more or less than $50k per year using some general data.

In [56]:
from fastai.tabular.all import *
from pathlib import Path
import pandas as pd

## Variables

In [57]:
modelName = 'stockScreenerV4.pkl'
trainingDataName = 'stockData.csv'
trainingFolder = Path.cwd().parent / 'TrainingData'
modelFolder = Path.cwd().parent.parent / 'TrainedModels'

# Training parameters
yNames = ['Future Year Change']
catNames = []
contNames = [
    'Open',
    'High', 
    'Low', 
    'Close', 
    'Volume', 
    'Dividends', 
    'Stock Splits', 
    'EV/EBIT', 
    'Market Cap', 
    'ROIC'
]

We can download a sample of this dataset with the usual untar_data command:

In [58]:
dataPath = Path()
dataPath.ls()

(#3) [Path('app.ipynb'),Path('stockFetcher.ipynb'),Path('tabular.ipynb')]

Then we can have a look at how the data is structured:

In [59]:
df = pd.read_csv(trainingFolder/trainingDataName)
df.head()

,Date,Ticker,Open,High,Low,Close,Volume,Dividends,Stock Splits,Future Year Change,EV/EBIT,Market Cap,ROIC
0,2019-01-17 00:00:00-05:00,AAPL,36.820325,37.646515,36.595868,37.216705,119284800,0.0,0.0,1.075702,10.460259,5.596611e+11,0.075524
1,2019-01-18 00:00:00-05:00,AAPL,37.608309,37.699047,37.245358,37.445938,135004000,0.0,0.0,1.049015,10.519029,5.631083e+11,0.075102
2,2019-01-22 00:00:00-05:00,AAPL,37.348034,37.424443,36.443045,36.605419,121576000,0.0,0.0,1.103545,10.303539,5.504686e+11,0.076673
3,2019-01-23 00:00:00-05:00,AAPL,36.808383,37.044779,36.223365,36.753464,92522400,0.0,0.0,1.105162,10.341494,5.526949e+11,0.076391
4,2019-01-24 00:00:00-05:00,AAPL,36.798839,36.887188,36.232925,36.462154,101766000,0.0,0.0,1.115865,10.266809,5.483142e+11,0.076947


Some of the columns are continuous (like age) and we will treat them as float numbers we can feed our model directly. Others are categorical (like workclass or education) and we will convert them to a unique index that we will feed to embedding layers. We can specify our categorical and continuous column names, as well as the name of the dependent variable in TabularDataLoaders factory methods:

In [60]:
dls = TabularDataLoaders.from_csv(trainingFolder/trainingDataName, path=dataPath, 
    y_names=yNames,
    cat_names=catNames,
    cont_names=contNames,
    procs = [Categorify, FillMissing, Normalize])

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/fastai/tabular/core.py:314: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  to[n].fillna(self.na_dict[n], inplace=True)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/fastai/tabular/core.py:314: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermed

The last part is the list of pre-processors we apply to our data:

* Categorify is going to take every categorical variable and make a map from integer to unique categories, then replace the values by the corresponding index.
* FillMissing will fill the missing values in the continuous variables by the median of existing values (you can choose a specific value if you prefer)
* Normalize will normalize the continuous variables (subtract the mean and divide by the std)

To further expose what’s going on below the surface, let’s rewrite this utilizing fastai’s TabularPandas class. We will need to make one adjustment, which is defining how we want to split our data. By default the factory method above used a random 80/20 split, so we will do the same:

In [61]:
splits = RandomSplitter(valid_pct=0.2)(range_of(df))

In [62]:
to = TabularPandas(df, procs=[Categorify, FillMissing,Normalize],
    y_names=yNames,
    cat_names = catNames,
    cont_names = contNames,
    splits=splits)

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/fastai/tabular/core.py:314: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  to[n].fillna(self.na_dict[n], inplace=True)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/fastai/tabular/core.py:314: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermed

Once we build our TabularPandas object, our data is completely preprocessed as seen below:

In [63]:
to.xs.iloc[:2]

,EV/EBIT_na,ROIC_na,Open,High,Low,Close,Volume,Dividends,Stock Splits,EV/EBIT,Market Cap,ROIC
90173,1,1,-0.394639,-0.396353,-0.394974,-0.397315,-0.275505,-0.065734,-0.010006,-0.157127,-0.401920,-0.002293
32338,1,1,6.200778,6.203207,6.187436,6.293496,-0.299221,-0.065734,-0.010006,-0.231090,-0.323947,-0.002243


Now we can build our DataLoaders again:

In [64]:
dls = to.dataloaders(bs=64)

The show_batch method works like for every other application:

In [65]:
dls.show_batch()

,EV/EBIT_na,ROIC_na,Open,High,Low,Close,Volume,Dividends,Stock Splits,EV/EBIT,Market Cap,ROIC,Future Year Change
0,False,False,26.093029,26.417511,25.839777,26.211737,2.241300e+06,-4.910658e-12,9.377977e-12,5.883828,3.295104e+10,0.134266,-0.507908
1,False,False,30.950000,31.339999,29.930000,30.540000,3.793300e+06,-4.910658e-12,9.377977e-12,5.441564,1.614992e+10,0.145179,4.351670
2,False,False,71.271405,71.722792,70.687509,71.703365,2.556200e+07,-4.910658e-12,9.377977e-12,6.529481,3.968064e+11,0.120990,0.682356
3,False,False,191.370117,192.092377,188.962586,189.646317,2.400000e+06,-4.910658e-12,9.377977e-12,25.428787,1.233169e+11,0.031067,-0.008084
4,False,False,269.540006,273.934998,269.540012,272.959996,4.179500e+06,-4.910658e-12,9.377977e-12,58.278989,2.736533e+11,0.013555,-0.657166
5,False,False,38.300002,39.410001,38.009995,38.810001,2.968860e+07,-4.910658e-12,9.377977e-12,13.657109,8.172260e+10,0.057845,-0.318475
6,False,False,34.242739,34.382119,33.702660,34.007543,2.849500e+07,-4.910658e-12,9.377977e-12,18.720413,1.354439e+11,0.042200,0.460549
7,False,False,148.963394,149.417419,146.800110,146.889145,1.934100e+06,-4.910658e-12,9.377977e-12,21.352978,5.985101e+10,0.036997,-0.150156
8,False,False,199.137512,199.724977,195.975053,196.895400,2.429200e+06,-4.910658e-12,9.377977e-12,26.259029,1.280306e+11,0.030085,0.115008
9,False,False,8.229162,8.324044,8.178554,8.235479,1.629070e+07,-4.910658e-12,9.377977e-12,3.164715,3.515498e+10,0.249628,-0.366185


We can define a model using the tabular_learner method. When we define our model, fastai will try to infer the loss function based on our y_names earlier.

Note: Sometimes with tabular data, your y’s may be encoded (such as 0 and 1). In such a case you should explicitly pass y_block = CategoryBlock in your constructor so fastai won’t presume you are doing regression.

In [66]:
learn = tabular_learner(dls, metrics=[rmse, mae])

And we can train that model with the fit_one_cycle method (the fine_tune method won’t be useful here since we don’t have a pretrained model).

In [67]:
learn.fit_one_cycle(25)

epoch,train_loss,valid_loss,_rmse,mae,time
0,0.600255,168.597015,12.984491,0.514245,00:17
1,0.562382,6.337741,2.517487,0.405817,00:16
2,0.786912,35406.183594,188.165314,2.809183,00:16
3,0.667987,83782.937500,289.453125,4.608423,00:16
4,0.558403,5260698.000000,2293.621582,27.050636,00:15
5,0.561352,4773.174805,69.088165,1.686867,00:15
6,0.480695,307020.875000,554.094971,8.304678,00:16
7,0.512904,5782519.000000,2404.687500,30.591057,00:16
8,0.459364,314314048.000000,17728.904297,243.735321,00:16
9,0.550267,98178464.000000,9908.504883,161.893860,00:16


We can then have a look at some predictions:

In [68]:
learn.show_results()

,EV/EBIT_na,ROIC_na,Open,High,Low,Close,Volume,Dividends,Stock Splits,EV/EBIT,Market Cap,ROIC,Future Year Change,Future Year Change_pred
0,1.0,1.0,0.146551,0.136998,0.131761,0.134939,-0.261831,-0.065734,-0.010006,0.277133,-0.391370,-0.002467,-0.196614,0.144419
1,1.0,1.0,-0.075261,-0.073756,-0.069064,-0.070247,-0.219157,-0.065734,-0.010006,-0.343630,0.025302,-0.002146,0.219566,0.208246
2,1.0,1.0,-0.132108,-0.124227,-0.130390,-0.124304,1.438915,-0.065734,-0.010006,1.118992,0.050713,-0.002601,-0.158213,0.306624
3,1.0,1.0,0.708160,0.695658,0.710715,0.711755,-0.291866,-0.065734,-0.010006,1.211563,-0.243788,-0.002610,0.328638,0.149170
4,1.0,1.0,-0.351547,-0.354336,-0.360706,-0.361192,-0.144883,-0.065734,-0.010006,-1.005574,-0.293984,-0.042700,-0.310169,0.514111
5,1.0,1.0,0.090498,0.087868,0.097534,0.088445,-0.257565,-0.065734,-0.010006,1.914854,-0.034580,-0.002657,-0.110707,0.031265
6,1.0,1.0,-0.111951,-0.107054,-0.119898,-0.106026,-0.239475,-0.065734,-0.010006,1.237969,-0.372352,-0.002612,-0.138276,0.342165
7,1.0,1.0,1.389679,1.369740,1.405800,1.390260,-0.280838,-0.065734,-0.010006,-0.737679,0.199638,-0.001138,0.628658,0.318232
8,1.0,1.0,-0.146209,-0.151565,-0.145054,-0.149377,-0.239651,-0.065734,-0.010006,0.119112,-0.142268,-0.002419,-0.277584,0.103195


## Export the model

In [69]:
learn.export(modelFolder / f'{modelName}')


To get prediction on a new dataframe, you can use the test_dl method of the DataLoaders. That dataframe does not need to have the dependent variable in its column.

In [70]:
from ipynb.fs.full.stockFetcher import getTickerData

In [72]:
predictionTarget = 'DNSKF'

test_df = getTickerData(predictionTarget)

# Ensure test_df is a DataFrame
if isinstance(test_df, dict):
	test_df = pd.DataFrame([test_df])

dl = learn.dls.test_dl(test_df)

prediction = learn.get_preds(dl=dl)
print(f"Predicted {yNames[0]} for {predictionTarget}:")
print(f"{prediction[0][0][0].item() * 100:.2f}%")

Static Data for DNSKF:
  Total Debt: 1576470970368
  Cash: 683344986112
  Shares Outstanding: 833260032
Approximated EBIT for DNSKF: 8934150144.0


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/fastai/tabular/core.py:314: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  to[n].fillna(self.na_dict[n], inplace=True)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/fastai/tabular/core.py:314: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermed

Predicted Future Year Change for DNSKF:
-1.45%


Note:
Since machine learning models can’t magically understand categories it was never trained on, the data should reflect this. If there are different missing values in your test data you should address this before training